<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [198]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Scrape Dataset

In [199]:
website_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Use beautiful soup library to scrape the table from Wikipedia

In [200]:
from bs4 import BeautifulSoup
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = urllib.request.urlopen(url)
article = req.read().decode()

with open('List of postal codes of Canada: M', 'w') as fo:
    fo.write(article)

In [201]:
from bs4 import BeautifulSoup

# Load article, turn into soup and get the <table>s.
article = open('List of postal codes of Canada: M').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')
#print(tables)
# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode','Borough', 'Neighborhood']:
        break

# Extract the columns we want and write to a semicolon-delimited text file.
with open('List of postal codes of Canada: M.txt', 'w') as fo:
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        postcode, borough, neighborhood = [td.text.strip() for td in tds[:3]]
        # Wikipedia does something funny with country names containing
        # accented characters: extract the correct string form.
        print('; '.join([postcode, borough, neighborhood]), file=fo)

In [202]:
#write the textfile to a Pandas dataframe
df = pd.read_csv('List of postal codes of Canada: M.txt', sep=';', 
                 converters={'Postcode': str, 'Borough': str, 'Neighborhood': str}, 
                 header=None, names=['Postcode','Borough', 'Neighborhood'])
print('Dimensions of the data frame is', df.shape)
df.head()

Dimensions of the data frame is (288, 3)


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Clean the dataset

Ignore cells with a borough that is Not assigned.

In [203]:
#Drop cells when Borough = Not assigned and show the final dimension of the array
df.drop(df[df.Borough == ' Not assigned'].index, inplace=True)
#reset the index to start from 0
df.index = np.arange(len(df))

Combine multiple rows into one row with the neighborhoods separated with a comma

Also, if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [204]:
df = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()

In [205]:
df.Neighborhood[85] = "Queen's Park"
df.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [206]:
print('Shape of the dataframe', df.shape)

Shape of the dataframe (103, 3)
